# Example 1: A Quick First Estimate

This notebook covers the most basic use case of the `NeuralMI` library: getting a single, quick estimate of mutual information between two variables.

**Goal:**
1.  Introduce the main `neural_mi.run` function.
2.  Use a simple dataset where the ground truth MI is known analytically.
3.  Compare our estimate to the ground truth to verify the library is working.

## 1. Imports

We'll need `torch` for data handling, our `run` function, and the data generator.

In [ ]:
import torch
import numpy as np
import neural_mi as nmi

## 2. Generating the Data

We will use the `generate_correlated_gaussians` function, which creates two multidimensional Gaussian variables, `X` and `Y`, with a precisely specified mutual information in **bits**.

The raw output of the generator is `(n_samples, n_features)`. However, our `ContinuousProcessor` expects the format `(n_channels, n_timepoints)`. For this simple case where each sample is a time point, we will treat features as channels and transpose the data accordingly.

In [ ]:
# --- Dataset Parameters ---
n_samples = 5000
dim = 5
ground_truth_mi_bits = 2.0

# --- Generate Raw 2D Data ---
# This creates data of shape (n_samples, dim).
x_raw, y_raw = nmi.datasets.generate_correlated_gaussians(
    n_samples=n_samples, 
    dim=dim, 
    mi=ground_truth_mi_bits
)

# Transpose to the expected (n_channels, n_timepoints) format for the processor
x_raw_transposed = x_raw.T
y_raw_transposed = y_raw.T

print(f"Transposed X data shape: {x_raw_transposed.shape}")
print(f"Transposed Y data shape: {y_raw_transposed.shape}")

## 3. Defining the Analysis Parameters

The `run` function requires a `base_params` dictionary. Since each sample is independent, we use a `window_size` of 1, which tells the processor to treat each time point (column) as a distinct sample.

In [ ]:
# The processor will treat each column as a sample.
processor_params = {'window_size': 1}

# Basic model and training parameters
base_params = {
    'n_epochs': 50, 'learning_rate': 1e-3, 'batch_size': 128,
    'patience': 5, 'embedding_dim': 16, 'hidden_dim': 64, 'n_layers': 2
}

## 4. Running the MI Estimation

Now we call the main `nmi.run` function. We add `random_seed=42` to ensure our result is reproducible.

In [ ]:
results = nmi.run(
    x_data=x_raw_transposed,  # Pass transposed data
    y_data=y_raw_transposed,  # Pass transposed data
    mode='estimate',
    processor_type='continuous',
    processor_params=processor_params,
    base_params=base_params,
    output_units='bits',
    random_seed=42 # For reproducibility
)

estimated_mi_bits = results.mi_estimate

print(f"\n--- Results ---")
print(f"Ground Truth MI:  {ground_truth_mi_bits:.3f} bits")
print(f"Estimated MI:     {estimated_mi_bits:.3f} bits")
print(f"Estimation Error: {abs(estimated_mi_bits - ground_truth_mi_bits):.3f} bits")

## 5. Conclusion

Success! The estimated MI is very close to the ground truth value we specified. We were able to get this estimate by formatting our data correctly for the `ContinuousProcessor`.